### 7.3 Pretrained Faster R-CNN Model

⚠️⚠️⚠️ *Please open this notebook in Google Colab* by click below link ⚠️⚠️⚠️<br><br>
<a href="https://colab.research.google.com/github/Muhammad-Yunus/Belajar-OpenCV-ObjectDetection/blob/main/Pertemuan%207/7.3%20pretrained_faster_rcnn.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><br><br><br>
- Click `Connect` button in top right Google Colab notebook,<br>
<img src="https://github.com/Muhammad-Yunus/Belajar-OpenCV-ObjectDetection/blob/main/Pertemuan%207/resource/cl-connect-gpu.png?raw=1" width="250px">
- If connecting process completed, it will turn to something look like this<br>
<img src="https://github.com/Muhammad-Yunus/Belajar-OpenCV-ObjectDetection/blob/main/Pertemuan%207/resource/cl-connect-gpu-success.png?raw=1" width="250px">

- Check GPU connected into Colab environment is active

In [ ]:
!nvidia-smi

In [60]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision import transforms
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2

- Define COCO Class Names (91 class)

In [ ]:
COCO_CLASS_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

- Load image Sample

In [35]:
# Download an example image from repo
import urllib
url, filename = ("https://github.com/Muhammad-Yunus/Belajar-OpenCV-ObjectDetection/raw/main/Pertemuan%207/image2.jpg", "image2.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [3]:
# show image using matplot lib
def imshow(image, isBGR=True):
    if isBGR :
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(image)
    plt.axis("off")

In [ ]:
# Load an example image and preprocess it
image_path = 'image2.jpg'  # Replace with your image path
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Convert image to tensor and normalize as required by ResNet
input_tensor = torch.tensor(image).float().permute(2, 0, 1).unsqueeze(0)
input_tensor = input_tensor / 255.0


imshow(image, isBGR=False)

- Load Pretrained Faster R-CNN with backbone ResNet-50
    - The pretrained model provoded by Torchvision 

In [ ]:
# Load the Faster R-CNN model pre-trained on COCO dataset from Torch Vision
model = fasterrcnn_resnet50_fpn_v2(
            weights='DEFAULT'
        )

# Set the model to evaluation mode
model.eval()

In [ ]:
# Perform inference
with torch.no_grad():
    predictions = model(input_tensor)

# Example: accessing the prediction results
print(predictions)  # Prints out the bounding boxes, labels, and scores


In [ ]:
# Filter predictions with a threshold score
threshold = 0.89
mask = predictions[0]['scores'] > threshold
filtered_boxes = predictions[0]['boxes'][mask]
filtered_scores = predictions[0]['scores'][mask]
filtered_labels = predictions[0]['labels'][mask]


# Draw bounding boxes, labels, and confidence scores on the image
image_with_box = image.copy()

for box, score, label in zip(filtered_boxes, filtered_scores, filtered_labels):
    x_min, y_min, x_max, y_max = map(int, box)  # Extract and convert box coordinates to int
    image_with_box = cv2.rectangle(
                            image_with_box, 
                            (x_min, y_min), 
                            (x_max, y_max), 
                            color=(0, 0, 255), 
                            thickness=2
                            )

    text = f"{COCO_CLASS_NAMES[label.item()]} : {score.item():.2f}"
    image_with_box = cv2.putText(
                            image_with_box, 
                            text, 
                            (x_min, y_min - 5), 
                            cv2.FONT_HERSHEY_SIMPLEX, 
                            0.8, 
                            (0, 255, 0), 
                            2
                            )

imshow(image_with_box, isBGR=False)
